# Grovers algorithm

In [1]:
#initialization
import matplotlib.pyplot as plt
import numpy as np
import math

# importing Qiskit
from qiskit import IBMQ, Aer, assemble, transpile
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.providers.ibmq import least_busy

# import basic plot tools
from qiskit.visualization import plot_histogram

from itertools import permutations

In [2]:

from copy import copy
# size = 4
# start_values = [(1,0,0),(2,0,1),(3,0,2),(4,0,3),(4,1,0),(3,1,1),(2,1,2),(1,1,3),(3,2,0),(1,2,1),(4,2,2),(2,2,3),(2,3,0),(4,3,1)]
# size = 3
# start_values = [(2,0,1),(3,0,2),(2,1,0),(1,1,2),(3,2,0)]
# size = 9
# start_values = [(4,0,0),(2,0,2),(3,1,4),(8,1,5),(9,2,1),(1,2,7),(8,2,8),(6,3,6),(1,3,8),(7,4,5),(5,4,6),(3,4,7),(1,5,3),(2,5,4),(5,6,4),(6,6,5),(1,6,6),(3,7,2),(9,7,3),(4,7,4),(2,8,0),(6,8,2),(8,8,4),(4,8,7),(7,8,8)]
print(len(start_values))
sudoku = np.zeros((size,size))
empty_places = []
for i in range(size):
    for j in range(size):
        empty_places.append((i,j))

for i in start_values[0:-1]:
    empty_places.remove((i[1],i[2]))
    sudoku[i[1]][i[2]] = i[0]

def create_list(currentlist,position, empty_places):
    if position >= len(empty_places):
        return currentlist
    newlist = []
    i = empty_places[position][0]
    j = empty_places[position][1]
    for su in currentlist:
        for value in range(1,size+1):
            newsudoku = copy(su)
            newsudoku[i][j] = value 
            newlist.append(newsudoku)
    return create_list(newlist, position+1,empty_places)

list_to_search = create_list([sudoku],0,empty_places)
print(len(list_to_search))
# 1024 was te lang voor 4x4



                

5
243


In [3]:
# Initialisation function 
def initialize_s(qc, qubits):
    """Apply a H-gate to 'qubits' in qc"""
    for q in qubits:
        qc.h(q)
    return qc


In [4]:
n_qubits = 1+int(math.log(len(list_to_search))/math.log(2))
qc = QuantumCircuit(n_qubits)
qc.h(range(n_qubits))
qc.draw()


┌───┐
q_0: ┤ H ├
     ├───┤
q_1: ┤ H ├
     ├───┤
q_2: ┤ H ├
     ├───┤
q_3: ┤ H ├
     ├───┤
q_4: ┤ H ├
     ├───┤
q_5: ┤ H ├
     ├───┤
q_6: ┤ H ├
     ├───┤
q_7: ┤ H ├
     └───┘

### Oracle for the list

Follow the equation: $U_{\omega} |{x}\rangle = (-1)^{f(x)} |{x}\rangle$ where $f(\omega) = -1$ and $f(x) = 1$ otherwise

In [5]:
# a function to create the oracle operator for a given target and a given list to search for max value 7
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.quantum_info.operators import Operator

def create_oracle(search_list):
    '''
    a function to create the oracle operator for a given target and a given list to search for max value 7
    target is the value which we want to find in the list
    search list is the list in which to search for the target
    '''
    def f(x):
        size = len(x)
        for i in range(0,size):
            for j in range(1,size+1):

                if np.count_nonzero(x[i] == j) > 1 or np.count_nonzero(x[:,i]==j) > 1:
                    return 1
        print(x)
        return -1

    size = int(math.pow(2,1+int(math.log(len(search_list))/math.log(2))))
    op = np.zeros([size,size])
    for i in range(size):
        if i < len(search_list):
            op[i,i] = f(search_list[i])
        else:
            op[i,i] = 1
    return Operator(op)
Oracle = create_oracle(list_to_search)
print(Oracle)

[[1. 2. 3.]
 [2. 3. 1.]
 [3. 1. 2.]]
Operator([[1.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 1.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 1.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
          ...,
          [0.+0.j, 0.+0.j, 0.+0.j, ..., 1.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 1.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 1.+0.j]],
         input_dims=(2, 2, 2, 2, 2, 2, 2, 2), output_dims=(2, 2, 2, 2, 2, 2, 2, 2))


In [6]:
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.quantum_info.operators import Operator

qc.append(Oracle, range(n_qubits))

# Diffusor


# qc.draw()

In [7]:
qc.h(range(n_qubits))
# qc.draw()

In [8]:
# here we create the general diffusion operator U_0:
size = int(2**(n_qubits))
diff_op = np.zeros((size,size))
for i in range(size):
    diff_op[i,i] = -1
diff_op[0,0] = 1
print(diff_op)
diff_op = Operator(diff_op)


[[ 1.  0.  0. ...  0.  0.  0.]
 [ 0. -1.  0. ...  0.  0.  0.]
 [ 0.  0. -1. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ... -1.  0.  0.]
 [ 0.  0.  0. ...  0. -1.  0.]
 [ 0.  0.  0. ...  0.  0. -1.]]


In [9]:

qc.append(diff_op,range(n_qubits))
qc.h(range(n_qubits))
# qc.draw()

In [10]:

qc.draw()

┌───┐┌──────────┐┌───┐┌──────────┐┌───┐
q_0: ┤ H ├┤0         ├┤ H ├┤0         ├┤ H ├
     ├───┤│          │├───┤│          │├───┤
q_1: ┤ H ├┤1         ├┤ H ├┤1         ├┤ H ├
     ├───┤│          │├───┤│          │├───┤
q_2: ┤ H ├┤2         ├┤ H ├┤2         ├┤ H ├
     ├───┤│          │├───┤│          │├───┤
q_3: ┤ H ├┤3         ├┤ H ├┤3         ├┤ H ├
     ├───┤│  Unitary │├───┤│  Unitary │├───┤
q_4: ┤ H ├┤4         ├┤ H ├┤4         ├┤ H ├
     ├───┤│          │├───┤│          │├───┤
q_5: ┤ H ├┤5         ├┤ H ├┤5         ├┤ H ├
     ├───┤│          │├───┤│          │├───┤
q_6: ┤ H ├┤6         ├┤ H ├┤6         ├┤ H ├
     ├───┤│          │├───┤│          │├───┤
q_7: ┤ H ├┤7         ├┤ H ├┤7         ├┤ H ├
     └───┘└──────────┘└───┘└──────────┘└───┘

In [11]:
# Excecute grovers algorithm for our test scenario:

number_of_repetitions = int(np.sqrt(2**n_qubits)*np.pi/4) # should be around sqrt(N)*pi/4
print(number_of_repetitions)
# intitialse the circuit:
circuit = QuantumCircuit(n_qubits)
circuit.h(range(n_qubits))

for i in range(number_of_repetitions):
   circuit.append(Oracle,range(n_qubits))
   circuit.h(range(n_qubits))
   circuit.append(diff_op,range(n_qubits))
   circuit.h(range(n_qubits))

circuit.measure_all()
# circuit.draw()
qc = circuit


12


In [12]:
aer_sim = Aer.get_backend('aer_simulator')
transpiled_circuit = transpile(qc, aer_sim)
qobj = assemble(transpiled_circuit)
results = aer_sim.run(qobj).result()
counts = results.get_counts()
plot_histogram(counts)
# index = 0
# for j in range(1,len(i)+1):
#     if i[-j] == '1':
#         index += 2**(j-1) 
# print(list_to_search[index])
# print(counts)
counts = sorted(counts, key=counts.get, reverse=True)
# print(counts)
for i in counts:
    index = 0
    for j in range(1,len(i)+1):
        if i[-j] == '1':
            index += 2**(j-1) 
    if index >= len(list_to_search):
        continue
    print(list_to_search[index])


[[1. 2. 3.]
 [2. 3. 1.]
 [3. 1. 2.]]


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=8e107b99-ec0e-43d9-aa18-8c6ee91759cf' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>